In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import sys

sys.path.append("../../../")

import getpass
import pickle
from pathlib import Path

import contextily as cx
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from povertymapping import nightlights, settings
from povertymapping.dhs import generate_dhs_cluster_level_data
from povertymapping.feature_engineering import (
    categorize_wealth_index,
    generate_features,
)
from povertymapping.iso3 import get_region_name
from povertymapping.rollout_grids import get_region_filtered_bingtile_grids

# Model Prediction on Rollout Grids: Malaysia

This notebook is the final step in the rollout and runs the final model to create relative wealth estimations over populated areas within the given country. The model predictions will have a spatial resolution of 2.4km.

The `predicted relative wealth` value gives us the relative wealth level of an area compared to the rest of the country, which fixes the value range from 0 (lowest wealth) to 1 (highest wealth). In between these extremes, each area's wealth estimate is scaled to a value between 0 and 1.

The predicted relative wealth value is later binned into 5 wealth categories A-E by dividing the distribution into quintiles (every 20th percentile).

## Set up Data Access
The following cell will prompt you to enter your EOG username and password. See [this page](https://eogdata.mines.edu/products/register/) to learn how to set-up your EOG account.

In [3]:
# Log-in using EOG credentials
username = os.environ.get("EOG_USER", None)
username = username if username is not None else input("Username?")
password = os.environ.get("EOG_PASSWORD", None)
password = password if password is not None else getpass.getpass("Password?")

# set save_token to True so that access token gets stored in ~/.eog_creds/eog_access_token
access_token = nightlights.get_eog_access_token(username, password, save_token=True)

2023-04-17 14:10:56.559 | INFO     | povertymapping.nightlights:get_eog_access_token:42 - Saving access_token to /home/alron/.eog_creds/eog_access_token.txt
2023-04-17 14:10:56.573 | INFO     | povertymapping.nightlights:get_eog_access_token:50 - Adding access token to environment var EOG_ACCESS_TOKEN


## Set country-specific parameters

In [4]:
COUNTRY_CODE = "my"
COUNTRY_OSM = "malaysia-singapore-brunei"
OOKLA_YEAR = 2019
NIGHTLIGHTS_YEAR = 2019

rollout_date = "-".join(os.getcwd().split("/")[-2].split("-")[:3])
rollout_grids_path = Path(f"./{rollout_date}-{COUNTRY_CODE}-rollout-grids.geojson")
rollout_grids_path

Path('2023-02-21-my-rollout-grids.geojson')

## Set Model Parameters

In [5]:
# Model to use for prediction
MODEL_SAVE_PATH = Path(f"../{rollout_date}-cross-country-model.pkl")

## Load Country Rollout AOI

The rollout area of interest is split into 2.4km grid tiles (zoom level 14), matching the areas used during model training. The grids are also filtered to only include populated areas based on Meta's High Resolution Settlement Layer (HRSL) data.

Refer to the previous notebook `2_my_generate_grids.ipynb` for documentation on generating this grid.

In [6]:
aoi = gpd.read_file(rollout_grids_path)
aoi.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 30202 entries, 0 to 30201
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   quadkey     30202 non-null  object  
 1   shapeName   30202 non-null  object  
 2   shapeISO    30202 non-null  object  
 3   shapeID     30202 non-null  object  
 4   shapeGroup  30202 non-null  object  
 5   shapeType   30202 non-null  object  
 6   pop_count   30202 non-null  float64 
 7   geometry    30202 non-null  geometry
dtypes: float64(1), geometry(1), object(6)
memory usage: 1.8+ MB


## Generate Features For Rollout AOI

If this is your first time running this notebook for this specific area, expect a long runtime for the following cell as it will download and cache the required datasets. It will then process the relevant features for each area specified. On subsequent runs, the runtime will be much faster as the data is already stored in your filesystem. 

In [7]:
%%time
scaler = MinMaxScaler
rollout_aoi = aoi.copy()

# Create features dataframe using generate_features module
features = generate_features(
    rollout_aoi,
    country_osm=COUNTRY_OSM,
    ookla_year=OOKLA_YEAR,
    nightlights_year=NIGHTLIGHTS_YEAR,
    scaled_only=False,
    sklearn_scaler=scaler,
    features_only=True,
    use_aoi_quadkey=True,
)

2023-04-17 14:10:59.517 | INFO     | povertymapping.osm:download_osm_country_data:198 - OSM Data: Cached data available for malaysia-singapore-brunei at /home/alron/.geowrangler/osm/malaysia-singapore-brunei? True
2023-04-17 14:10:59.518 | DEBUG    | povertymapping.osm:load_pois:160 - OSM POIs for malaysia-singapore-brunei being loaded from /home/alron/.geowrangler/osm/malaysia-singapore-brunei/gis_osm_pois_free_1.shp
2023-04-17 14:11:11.067 | INFO     | povertymapping.osm:download_osm_country_data:198 - OSM Data: Cached data available for malaysia-singapore-brunei at /home/alron/.geowrangler/osm/malaysia-singapore-brunei? True
2023-04-17 14:11:11.068 | DEBUG    | povertymapping.osm:load_roads:179 - OSM Roads for malaysia-singapore-brunei being loaded from /home/alron/.geowrangler/osm/malaysia-singapore-brunei/gis_osm_roads_free_1.shp
2023-04-17 14:12:43.204 | DEBUG    | povertymapping.ookla:load_type_year_data:79 - Contents of data cache: []
2023-04-17 14:12:43.205 | INFO     | povert

CPU times: user 3min 36s, sys: 11.3 s, total: 3min 47s
Wall time: 3min 48s


In [8]:
# Save raw features, can be used for validation
raw_features = features[[col for col in features.columns if "_scaled" not in col]]
# Then keep only scaled columns
features = features[[col for col in features.columns if "_scaled" in col]]

## Inspect the generated features

In [9]:
features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30202 entries, 0 to 30201
Data columns (total 61 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   poi_count_scaled                          30202 non-null  float64
 1   atm_count_scaled                          30202 non-null  float64
 2   atm_nearest_scaled                        30202 non-null  float64
 3   bank_count_scaled                         30202 non-null  float64
 4   bank_nearest_scaled                       30202 non-null  float64
 5   bus_station_count_scaled                  30202 non-null  float64
 6   bus_station_nearest_scaled                30202 non-null  float64
 7   cafe_count_scaled                         30202 non-null  float64
 8   cafe_nearest_scaled                       30202 non-null  float64
 9   charging_station_count_scaled             30202 non-null  float64
 10  charging_station_nearest_scaled   

## Run Model on AOI

### Load Model

In [10]:
with open(MODEL_SAVE_PATH, "rb") as f:
    model = pickle.load(f)

### Make Predictions

In [11]:
rollout_aoi["Predicted Relative Wealth Index"] = model.predict(features.values)

## Binning predictions into wealth categories

Afterwards, we label the predicted relative wealth by binning them into 5 categories: `A`, `B`, `C`, `D`, and `E` where `A` is the highest and `E` is the lowest. 

We can create these wealth categories by splitting the output `Predicted Relative Wealth Index` distribution into 5 equally sized **quintiles**, i.e. every 20th percentile. 

This categorization may be modified to suit the context of the target country.

In [12]:
rollout_aoi["Predicted Wealth Category (quintile)"] = categorize_wealth_index(
    rollout_aoi["Predicted Relative Wealth Index"]
).astype(str)

In [13]:
rollout_aoi.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 30202 entries, 0 to 30201
Data columns (total 10 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   quadkey                               30202 non-null  object  
 1   shapeName                             30202 non-null  object  
 2   shapeISO                              30202 non-null  object  
 3   shapeID                               30202 non-null  object  
 4   shapeGroup                            30202 non-null  object  
 5   shapeType                             30202 non-null  object  
 6   pop_count                             30202 non-null  float64 
 7   geometry                              30202 non-null  geometry
 8   Predicted Relative Wealth Index       30202 non-null  float64 
 9   Predicted Wealth Category (quintile)  30202 non-null  object  
dtypes: float64(2), geometry(1), object(7)
memory usage: 2.3+ MB


In [14]:
rollout_aoi.head(2)

,quadkey,shapeName,shapeISO,shapeID,shapeGroup,shapeType,pop_count,geometry,Predicted Relative Wealth Index,Predicted Wealth Category (quintile)
0,13223202222021,Klang,None,MYS-ADM2-3_0_0-B57,MYS,ADM2,1948.905696,"POLYGON ((101.27197 2.92110, 101.27197 2.94304...",0.366662,B
1,13223202222023,Klang,None,MYS-ADM2-3_0_0-B57,MYS,ADM2,5633.984424,"POLYGON ((101.27197 2.89915, 101.27197 2.92110...",0.412123,A


## Save output

In [15]:
%%time
rollout_aoi.to_file(
    f"{rollout_date}-{COUNTRY_CODE}-rollout-output.geojson",
    driver="GeoJSON",
    index=False,
)

CPU times: user 9.16 s, sys: 140 ms, total: 9.3 s
Wall time: 9.31 s


In [ ]:
# Join back raw features and save
rollout_output_with_features = rollout_aoi.join(raw_features).join(features)
rollout_output_with_features.to_file(
    f"{rollout_date}-{COUNTRY_CODE}-rollout-output-with-features.geojson",
    driver="GeoJSON",
    index=False,
)

## Visualizations

### Inspect predicted wealth index and output dataframe

In [ ]:
rollout_aoi[["Predicted Relative Wealth Index"]].hist()

### Create Static Maps
#### Plot Predicted Relative Wealth Index

In [ ]:
plt.cla()
plt.clf()
rollout_aoi_plot = rollout_aoi.to_crs("EPSG:3857")
ax = rollout_aoi_plot.plot(
    "Predicted Relative Wealth Index",
    figsize=(20, 8),
    cmap="viridis",
    legend=True,
    legend_kwds={"shrink": 0.8},
)
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)
ax.set_axis_off()
plt.title("Predicted Relative Wealth Index")
plt.tight_layout()
plt.savefig(f"{rollout_date}-{COUNTRY_CODE}-predicted-wealth-index.png")
plt.show()

#### Plot Predicted Relative Wealth Index Category

In [ ]:
plt.cla()
plt.clf()
rollout_aoi_plot = rollout_aoi.to_crs("EPSG:3857")
ax = rollout_aoi_plot.plot(
    "Predicted Wealth Category (quintile)",
    figsize=(20, 8),
    cmap="viridis_r",
    legend=True,
)
cx.add_basemap(ax, source=cx.providers.OpenStreetMap.Mapnik)
ax.set_axis_off()
plt.title("Predicted Wealth Category")
plt.tight_layout()
plt.savefig(f"{rollout_date}-{COUNTRY_CODE}-predicted-wealth-bin.png")
plt.show()

### Create an Interactive Map

In [ ]:
cols_of_interest = [
    "quadkey",
    "shapeName",
    "shapeGroup",
    "pop_count",
    "avg_rad_mean",
    "mobile_2019_mean_avg_d_kbps_mean",
    "fixed_2019_mean_avg_d_kbps_mean",
    "poi_count",
    "road_count",
    "Predicted Relative Wealth Index",
    "Predicted Wealth Category (quintile)",
]

# Warning: This can be a bit laggy due to the large amount of tiles being visualized

# Uncomment the ff if you want to viz the raw wealth predictions
# rollout_aoi.explore(column='Predicted Relative Wealth Index', tooltip=cols_of_interest, cmap="viridis")

# Uncomment the ff if you want to view the quintiles
# rollout_aoi.explore(column='Predicted Wealth Category (quintile)', tooltip=cols_of_interest, cmap="viridis_r")

Alternatively, you may also try to visualize this interactively in [Kepler](https://kepler.gl/demo) by uploading the rollout output geojson file.